# Data Loading & Cleaning 

## Set Up

In [2]:
import pandas as pd 
import numpy as np 
import random
import os
import matplotlib.pyplot as plt
import seaborn as sns

## Pre-Written Functions

### DataFrame Summary

In [3]:
def dataframe_summary(df: pd.DataFrame): 
    """
    Creating summary report of a DataFrame.
    
    Parameters:
    df (pd.DataFrame): Dataframe

    Returns:
    pd.DataFrame: A summary table containing key info on required for cleaning and analysis.
    """
    summary = {
        "Total Rows": df.shape[0],
        "Total Columns": df.shape[1],
        "Total Missing Values": df.isnull().sum().sum(),
        "Columns with Missing Values": (df.isnull().sum() > 0).sum(),
        "Duplicate Rows": df.duplicated().sum(),
        "Numerical Columns": df.select_dtypes(include=["number"]).shape[1],
        "Categorical Columns": df.select_dtypes(include=["object", "category"]).shape[1],
    }

    # Create missing value report 
    missing_values = df.isnull().sum()
    missing_values = missing_values[missing_values > 0].sort_values(ascending=False)

    # Specifying data types for each column
    column_data_types = df.dtypes.astype(str)

    
    print("=" * 60)
    #title
    print("Dataframe Report")
    #end section
    print("=" * 60)
    
    for key, value in summary.items():
        print(f"{key}: {value}")
    
    print("\nMissing Values per Column:")
    if not missing_values.empty:
        print(missing_values)
    else:
        print("No missing values found.")

    print("\nData Types per Column:")
    print(column_data_types)

    print("\nDescriptive Statistics (Numerical Columns):")
    print(df.describe().T)

   
    print("=" * 60)

    # create summary as DataFrame for reference
    summary_df = pd.DataFrame(summary, index=["Value"]).T
    column_types_df = pd.DataFrame(column_data_types, columns=["Data Type"])

    # add column data types
    final_summary = summary_df.merge(column_types_df, left_index=True, right_index=True, how="outer")

    return final_summary

# Example usage:
# summary_df = dataframe_summary(df)

## Data Dictionary

- `Title`: Vehicle Model Name  

- `Price`: OTR price of the Vehicle Listed 

- `Mileage(miles)`: Vehicle's recorded mileage since ownership 

- `Registration_Year`: Year of vehicle production

- `Previous Owners`: Number of owners the vehicle has had by time of sale price being logged 

- `Fuel Type`: Vehicle fuel type indicatice also of powertrain 

- `Body type`: Body type of vehicle listed 

- `Engine`: Size of vehicle engine 

- `Gearbox`: Vehicle transmission 

- `Doors`: Number of doors on the vehicle

- `Seats`: Number of seats in the vehicle 

- `Emission Class`: Emission Class of the Vehicle 

- `Service history`: Service history at time of sale price being logged 



## The Data 

The dataset contains data sourced from Autotrader UK a leading automotive marketplace website. There are 3,685 data points each representing a unique vehicle listing and distinct features.

Our aim is to use analysis and modelling tools to create actionable insights on feature importance in remarketing value. Allowing remarketing companies to predict vehicle sale value and decide when best to remarket vehicles to maximise return.

In [6]:
df = pd.read_csv("Data/used_cars_UK.csv", index_col= 0)
df.head()

,title,Price,Mileage(miles),Registration_Year,Previous Owners,Fuel type,Body type,Engine,Gearbox,Doors,Seats,Emission Class,Service history
0,SKODA Fabia,6900,70189,2016,3.0,Diesel,Hatchback,1.4L,Manual,5.0,5.0,Euro 6,NaN
1,Vauxhall Corsa,1495,88585,2008,4.0,Petrol,Hatchback,1.2L,Manual,3.0,5.0,Euro 4,Full
2,Hyundai i30,949,137000,2011,NaN,Petrol,Hatchback,1.4L,Manual,5.0,5.0,Euro 5,NaN
3,MINI Hatch,2395,96731,2010,5.0,Petrol,Hatchback,1.4L,Manual,3.0,4.0,Euro 4,Full
4,Vauxhall Corsa,1000,85000,2013,NaN,Diesel,Hatchback,1.3L,Manual,5.0,5.0,Euro 5,NaN


In [7]:
summary_df = dataframe_summary(df)

Dataframe Report
Total Rows: 3685
Total Columns: 13
Total Missing Values: 4746
Columns with Missing Values: 6
Duplicate Rows: 826
Numerical Columns: 6
Categorical Columns: 7

Missing Values per Column:
Service history    3145
Previous Owners    1409
Emission Class       87
Engine               45
Seats                35
Doors                25
dtype: int64

Data Types per Column:
title                 object
Price                  int64
Mileage(miles)         int64
Registration_Year      int64
Previous Owners      float64
Fuel type             object
Body type             object
Engine                object
Gearbox               object
Doors                float64
Seats                float64
Emission Class        object
Service history       object
dtype: object

Descriptive Statistics (Numerical Columns):
                    count          mean           std     min      25%  \
Price              3685.0   5787.145726   4480.810572   400.0   2490.0   
Mileage(miles)     3685.0  81328.

## Missing Values 

Missing values are present in:

- Service history    (3145)

- Previous Owners    (1409)

- Emission Class       (87)

- Engine               (45)

- Seats                (35)

- Doors                (25)

As emission class, engine, seats and doors have fewer missing values these . 

As Service history and has a large proportion (~85%) unpopulated we create a binary feature showing wether service history is declared on the vehicle listing.

Previous Owners is missing ~38%. this is significant but the feature is important to preserve so it will be imputed. 


### Imputing Missing Values 

#### Binary Column for Service History

In [8]:

# creating binary feature, service history included in decsription
df['Has_Service_History'] = df['Service history'].notna().astype(int)


In [12]:
# Dropping the original service history column
df = df.drop(columns=["Service history"])

#### Imputing Numerical Categories 

In [15]:
# Filling missing numerical values with median 

df["Previous Owners"].fillna(df["Previous Owners"].median(), inplace=True)
df["Doors"].fillna(df["Doors"].median(), inplace=True)
df["Seats"].fillna(df["Seats"].median(), inplace=True)

#### Imputing Categorical Categories 

In [26]:
# Filling missing values with median

df['Previous Owners'] = df['Previous Owners'].fillna(df['Previous Owners'].median())

# Converting to integer
 
df["Previous Owners"] = df["Previous Owners"].astype('Int64')